In [ ]:
#### 모델 로드 하기 ###
import cv2, torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import models
import numpy as np

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256))
])

In [ ]:
class suffle(nn.Module) :
    def __init__(self, output) :
        super().__init__()
        self.model = models.shufflenet_v2_x0_5(pretrained=True)
        self.model.fc = torch.nn.Linear(1024, output) #change the last FC layer
        
    def forward(self, x) :
        output = self.model(x)
        return output

In [ ]:
# 전체 모델 가져옴
model = torch.load('C:\data\\finger\ShuffleNet_total_model.pt', map_location=torch.device('cpu')) 
model.eval() # 평가 모드?

In [ ]:
# 가중치만 가져옴
#model = suffle(6) # 6개의 class
#model.load_state_dict(torch.load('C:\data\ShuffleNet_model.pt', map_location=torch.device('cpu'))) # 저장해논 path에서 가중치 불러옴
#model.eval() # 평가 모드?

In [ ]:
print(model.state_dict()) # 가중치 확인

In [ ]:
### 모델 테스트 ###
import PIL.Image as pilimg
import matplotlib.pyplot as plt

path = 'C:\data\\finger\\img111.png' # 사진 경로
image = cv2.imread(path) # 기존 fingers 사진일 경우 cv2.imread(path)를 하면 색갈이 제대로 나옴
plt.imshow(image)

In [ ]:
image = transform(image)
image

In [ ]:
image.shape

In [ ]:
image = image.unsqueeze(0) # 0번째에 차원을 하나 늘려줌
image.shape

In [ ]:
pred = nn.functional.softmax(model(image), dim=1)
torch.set_printoptions(sci_mode=False)

pred

In [ ]:
pred = pred.detach().numpy()
top_1 = np.argmax(pred, axis=1)
score = np.amax(pred, axis=1)
print(top_1, score)

In [ ]:
##### webcam test ##### 
import cv2 
cap = cv2.VideoCapture(0) 
cap.set(3, 720) 
cap.set(4, 1080) 


while True: 
    ret, frame = cap.read()
    frame = cv2.flip(frame,1) # 좌우반전
    cv2.imshow('test', frame) 
    
    if cv2.waitKey(1) & 0xFF == ord('q'): # 해당 창을 클릭 후 q 누르면 종료(안되면 영문 바꿔서? 그래도 안되면 컨트롤알트딜리트..)
        break
        
cap.release() 
cv2.destroyAllWindows()

In [ ]:
### webcam을 이용한 실시간 예측 ###
import PIL 
import cv2
import torchvision 

In [ ]:
def preprocess(image): # 예측전 이미지 전처리
    image = PIL.Image.fromarray(image) #Webcam frames are numpy array format
                                       #Therefore transform back to PIL image
    #print(image)                             
    image = transform(image)
    image = image.float()
    #image = image.cpu() # 원래 코드 : image = image.cuda() (gpu가 없기 떄문에)
    image = image.unsqueeze(0) # 3d -> 4d
    return image                            #dimension out of our 3-D vector Tensor

In [ ]:
cap = cv2.VideoCapture(0) #Set the webcam
cap.set(3,900)
cap.set(4,600)

fps = 0

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while True:
    ret, frame = cap.read() #Capture each frame
    frame = cv2.flip(frame,1)
    
    if fps == 4:
        image   = frame[150:400, 300:600] # [높이범위, 폭범위] -> 밑에 rectangular 범위와 맞춰줘야함
        image   = preprocess(image)
        prediction   = model(image)
        fps = 0
        
        pred = nn.functional.softmax(prediction, dim=1)
        
    fps += 1
    cv2.putText(frame, '0 = %.3f' %(pred[0][0]),(700,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, '1 = %.3f' %(pred[0][1]),(700,230), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, '2 = %.3f' %(pred[0][2]),(700,210), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, '3 = %.3f' %(pred[0][3]),(700,190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, '4 = %.3f' %(pred[0][4]),(700,170), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, '5 = %.3f' %(pred[0][5]),(700,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.rectangle(frame,(300,150),(600,400), (0,0,250), 2) # 박스 치는 코드 (완쪽 아래 꼭짓점, 오른쪽 위 꼭짓점, (B,G,R), 두께)
    cv2.imshow("finger detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()